In [1]:
import tensorflow as tf
import numpy as np

In [12]:
n_inputs = 28 * 28
n_hidden1 = 200
n_hidden2 = 100
n_output = 10

In [13]:
tf.reset_default_graph()
X = tf.placeholder(dtype=tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(dtype=tf.int64, shape=(None), name='y')

In [14]:
from tensorflow.contrib.layers import fully_connected

In [15]:
with tf.name_scope('dnn'):
    hidden1 = fully_connected(X, n_hidden1, scope='hidden1')
    hidden2 = fully_connected(hidden1, n_hidden2, scope='hidden2')
    logits = fully_connected(hidden2, n_output, scope='outputs', activation_fn=None)

In [16]:
with tf.name_scope('loss'):
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(cross_entropy)

In [17]:
learning_rate = 0.01

with tf.name_scope('train'):
        optmizer = tf.train.GradientDescentOptimizer(learning_rate)
        train_op = optmizer.minimize(loss)

In [18]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [19]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [20]:
n_epochs = 40
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(train_op, feed_dict={X: X_batch, y: y_batch})
            
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Train accuracy: 0.9 Test accuracy: 0.897
1 Train accuracy: 0.92 Test accuracy: 0.9152
2 Train accuracy: 0.86 Test accuracy: 0.9263
3 Train accuracy: 0.84 Test accuracy: 0.9326
4 Train accuracy: 0.92 Test accuracy: 0.9388
5 Train accuracy: 0.98 Test accuracy: 0.9409
6 Train accuracy: 0.94 Test accuracy: 0.946
7 Train accuracy: 0.96 Test accuracy: 0.9482
8 Train accuracy: 0.96 Test accuracy: 0.9506
9 Train accuracy: 0.96 Test accuracy: 0.9541
10 Train accuracy: 0.98 Test accuracy: 0.9566
11 Train accuracy: 0.98 Test accuracy: 0.9587
12 Train accuracy: 0.96 Test accuracy: 0.9592
13 Train accuracy: 1.0 Test accuracy: 0.9605
14 Train accuracy: 0.96 Test accuracy: 0.9629
15 Train accuracy: 0.96 Test accuracy: 0.9639
16 Train accuracy: 0.94 Test accuracy: 0.9642
17 Train accuracy: 0.96 Test accuracy: 0.9656
18 Train accuracy: 0.96 Test accuracy: 0.9671
19 Train accuracy: 0.96 Test accuracy: 0.9677
20 Train accuracy: 0.98 Test accuracy: 0.9684
21 Train accuracy: 0.98 Test accuracy: 0.9696
22

In [11]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = b"<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

show_graph(tf.get_default_graph())